<a href="https://colab.research.google.com/github/Itsuki-Hamano123/experiment_search_engine/blob/master/elmo_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import datetime
from pprint import pprint
import pickle
import time
import numpy as np
import pandas as pd

## 形態素解析ツール用意
https://qiita.com/Fulltea/items/90f6ebe6dcceaf64eaef

In [0]:
# mecabのインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

In [0]:
# Neologd辞書のインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [0]:
import MeCab
import subprocess

In [4]:
!ls

drive  ELMoForManyLangs  mecab-ipadic-neologd  sample_data


In [39]:
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
# Neologd辞書の設定
m_neologd = MeCab.Tagger("-d {0}".format(neologd_path))

m_neologd_sample_txt = "8月3日に放送された「中居正広の金曜日のスマイルたちへ」(TBS系)で、1日たった5分でぽっこりおなかを解消するというダイエット方法を紹介。キンタロー。のダイエットにも密着。"

print("Mecab ipadic NEologd:\n",m_neologd.parse(m_neologd_sample_txt))

ERROR! Session/line number was not unique in database. History logging moved to new session 63
Mecab ipadic NEologd:
 8月3日	名詞,固有名詞,一般,*,*,*,8月3日,ハチガツミッカ,ハチガツミッカ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
放送	名詞,サ変接続,*,*,*,*,放送,ホウソウ,ホーソー
さ	動詞,自立,*,*,サ変・スル,未然レル接続,する,サ,サ
れ	動詞,接尾,*,*,一段,連用形,れる,レ,レ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
「	記号,括弧開,*,*,*,*,「,「,「
中居正広の金曜日のスマイルたちへ	名詞,固有名詞,一般,*,*,*,中居正広の金曜日のスマイルたちへ,ナカイマサヒロノキンヨウビノスマイルタチヘ,ナカイマサヒロノキンヨービノスマイルタチヘ
」(	記号,一般,*,*,*,*,*
TBS	名詞,固有名詞,一般,*,*,*,TBS,ティービーエス,ティービーエス
系	名詞,接尾,一般,*,*,*,系,ケイ,ケイ
)	記号,一般,*,*,*,*,*
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
、	記号,読点,*,*,*,*,、,、,、
1日	名詞,固有名詞,一般,*,*,*,1日,ツイタチ,ツイタチ
たった	副詞,助詞類接続,*,*,*,*,たった,タッタ,タッタ
5分	名詞,固有名詞,一般,*,*,*,5分,ゴフン,ゴフン
で	助詞,格助詞,一般,*,*,*,で,デ,デ
ぽっこりおなか	名詞,固有名詞,一般,*,*,*,ぽっこりおなか,ポッコリオナカ,ポッコリオナカ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
解消	名詞,サ変接続,*,*,*,*,解消,カイショウ,カイショー
する	動詞,自立,*,*,サ変・スル,基本形,する,スル,スル
という	助詞,格助詞,連語,*,*,*,という,トイウ,トユウ
ダイエット方法	名詞,固有名詞,一般,*,*,*,ダイエット方法,ダイエットホウホウ,ダイエットホウホー
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
紹介	名詞,サ変接続,*,*,*,*,紹介,ショウカイ,ショーカイ
。	記号,句点,*,*,*,*,。,。,。
キンタ

## ELMoモデルの準備
ELMoForManyLangsを使う

In [0]:
# ELMoForManyLangsに必要なライブラリ
!pip install allennlp

### ELMoForManyLangsを利用
https://github.com/HIT-SCIR/ELMoForManyLangs#use-elmoformanylangs-programmatically

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
cd drive/My Drive/機械学習練習/全文検索エンジン作成

/content/drive/My Drive/機械学習練習/全文検索エンジン作成


In [11]:
!ls

build	 elmoformanylangs	    embedding		  setup.py
configs  elmoformanylangs.egg-info  mecab-ipadic-neologd
dist	 elmo_search_engine.ipynb   README.md


In [0]:
# ELMoForManyLangsのセットアップ
!python setup.py install

In [0]:
from elmoformanylangs import Embedder

In [0]:
def time_watch(func):
    """時間計測用デコレータ関数
    
    Parameters
    -----
    func : function
        実行したい関数

    """
    import functools
    import datetime
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.today()
        result = func(*args, **kwargs)
        end = datetime.datetime.today()
        print("実行時間:{time}".format(time=end - start))
        return result
    return wrapper

### 日本語の事前学習済み重みのダウンロード
embeddingディレクトに格納した<br>
https://qiita.com/mkt3/items/9577b63900109ff91665


In [0]:
# モデルのディレクトリ
MODEL_DIR = './embedding/word_unit_embedding_model/'

In [16]:
elmo_model = Embedder(MODEL_DIR)

2020-03-29 07:33:28,934 INFO: word embedding size: 387179
2020-03-29 07:33:42,783 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(387179, 100, padding_idx=3)
    )
    (projection): Linear(in_features=100, out_features=512, bias=True)
  )
  (encoder): ElmobiLm(
    (forward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, out_features=16384, bias=True)
      (state_projection): Linear(in_features=4096, out_features=512, bias=False)
    )
    (backward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, out_features=16384, bias=True)
      (state_projection): Linear(in_features=4096, out_features=512, bias=False)
    )
    (forward_layer_1): LstmCellWithProjection(
      (input_linearity): Line

### 入力文書の前処理を定義

In [0]:
def is_stop_word(pos):
    """
    品詞からストップワードか判断

    Parameters
    -----
    pos : string
        品詞

    Returns
    -----
    stop_word_flag : boolean
    """
    stop_word_list = ["助詞", "助動詞", "記号", "BOS/EOS"]
    stop_word_flag = True
    if pos in stop_word_list:
        stop_word_flag = False
    return stop_word_flag

def split_text(text):
    """
    文章をわかち書きする

    Parameters
    -----
    text : string

    Returns
    -----
    result : list of string
    """
    result = []
    node = m_neologd.parseToNode(text)
    while node:
        if is_stop_word(node.feature.split(",")[0]):
            result.append(node.surface)
        node = node.next
    return result

In [0]:
def preprocessing_documents(documents):
    """
    文書をELMo用に前処理

    Parameters
    -----
    documents : list of string

    Returns
    -----
    results : list of list of string
    """
    results = []
    # バグ対処
    m_neologd.parse('')
    for document in documents:
        result = split_text(document)
        results.append(result)
    return results

In [0]:
elmo_sample_docments = ["私は犬", "犬は私", "今日の天気は雪のち晴れです"]

In [73]:
proc_documents = preprocessing_documents(elmo_sample_docments)
proc_documents

[['私', '犬'], ['犬', '私'], ['今日の天気', '雪', 'のち', '晴れ']]

### ELMoで単語の分散表現を予測

In [0]:
@time_watch
def predict_word_vec(model, proc_documents):
    """
    モデルを使い単語の分散表現を予測

    Parmeters
    -----
    model : model_obj
    proc_documents : proc_documents_obj
    
    Returns
    -----
    pred_vector : list of nparray
    """
    pred_vector = model.sents2elmo(proc_documents)
    return pred_vector

In [77]:
documnets_words_vec = predict_word_vec(elmo_model, proc_documents)

2020-03-29 09:00:18,877 INFO: 1 batches, avg len: 4.7


実行時間:0:00:00.045184


In [105]:
print(documnets_words_vec[0].shape)
print(type(documnets_words_vec[0]))
documnets_words_vec

(2, 1024)
<class 'numpy.ndarray'>


[array([[-0.5977612 ,  1.4001379 , -1.3015023 , ..., -0.14499956,
          0.27046227,  0.00478347],
        [-1.8453506 ,  1.2980078 , -1.2131561 , ...,  0.302079  ,
         -0.14396201, -0.2849956 ]], dtype=float32),
 array([[-1.8625396e+00,  1.3427652e+00, -7.6916867e-01, ...,
          1.6443498e-01, -1.6299309e-01, -1.3014723e-01],
        [-1.8234619e+00,  1.4184309e+00, -8.1920910e-01, ...,
         -1.5965501e-03,  3.2706022e-02,  2.2069255e-02]], dtype=float32),
 array([[-0.33430147,  1.3464347 ,  0.26074216, ..., -0.05496437,
          0.18364926,  0.06471634],
        [-1.285893  ,  1.1447811 ,  0.2148868 , ...,  0.18514585,
          0.30238673, -0.04076589],
        [-1.1255991 ,  1.2842447 , -0.35377732, ...,  0.17827515,
          0.61796623,  0.00764098],
        [-0.9844549 ,  1.1928868 , -0.10221224, ..., -0.10837878,
          0.09892287,  0.20971125]], dtype=float32)]

#### 実行結果
「私は犬」と「犬は私」の２つの文章において、それぞれの文章で「私」「犬」は文章内での順番が考慮され、別々の分散表現が得られていることが分かる。

### 単語ベクトルから文書ベクトルを算出
今回は簡易的に、単語ベクトルの合計値を文書ベクトルする<br>

In [0]:
@time_watch
def calc_doc_vec(documnets_words_vec):
    """
    単語ベクトルから文書ベクトルを計算

    Parameters
    -----
    documents_word_vec : list of nparray

    Returns
    -----
    documents_vec : ndarray(文書数, 1つの文書内の次元数)
    """
    documnets_vec = []
    for document_words_vec in documnets_words_vec:
        document_vec = np.sum(document_words_vec, axis=0)
        documnets_vec.append(document_vec)

    documnets_vec = np.array(documnets_vec)
    return documnets_vec

In [220]:
documnets_vec = calc_doc_vec(documnets_words_vec=documnets_words_vec)
documnets_vec

実行時間:0:00:00.000525


array([[-2.443112  ,  2.6981459 , -2.5146585 , ...,  0.15707943,
         0.12650026, -0.2802121 ],
       [-3.6860015 ,  2.7611961 , -1.5883777 , ...,  0.16283843,
        -0.13028707, -0.10807798],
       [-3.7302485 ,  4.968347  ,  0.01963941, ...,  0.20007786,
         1.2029251 ,  0.24130267]], dtype=float32)

In [222]:
print(documnets_vec.shape)
documnets_vec

(3, 1024)


array([[-2.443112  ,  2.6981459 , -2.5146585 , ...,  0.15707943,
         0.12650026, -0.2802121 ],
       [-3.6860015 ,  2.7611961 , -1.5883777 , ...,  0.16283843,
        -0.13028707, -0.10807798],
       [-3.7302485 ,  4.968347  ,  0.01963941, ...,  0.20007786,
         1.2029251 ,  0.24130267]], dtype=float32)

## faissのインストール
参考：https://qiita.com/SakuraDk/items/0c03b7ebd4aef8d6caed#colaboratory%E4%B8%8A%E3%81%A7%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB

In [0]:
# ダウンロードは一度のみ実行で大丈夫
#!wget https://anaconda.org/pytorch/faiss-gpu/1.4.0/download/linux-64/faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2
#!tar xvjf faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2

In [122]:
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [0]:
import faiss

## 検索エンジン作成

### 検索対象となるドキュメント読み込み

In [153]:
DATA_DIR = "./datasets"
DATA_PATH = "a_ntcir_all_data.csv"
TEXT_COLUMN = "text"

headers = [TEXT_COLUMN]
register_documents = pd.read_csv(DATA_DIR+"/"+DATA_PATH, names=headers)
print("データ件数:{size}".format(size=len(register_documents)))
register_documents.index = register_documents.index + 1
register_documents.head()

データ件数:4125


,text
1,日本選手としては村上雅則（ジャイアンツ）、野茂英雄（ドジャース）、鈴木誠（マリナーズ）、長谷...
2,米大リーグのニューヨーク・メッツは１３日、フリーエージェント（ＦＡ）宣言した前ヤクルトの吉井...
3,竹下氏流の「広く浅く」という資金集めは、リクルート疑惑さなかの８９年に自殺した同氏の元秘書、...
4,また、リクルート事件さなかの八九年四月にも、退陣を表明したばかりの竹下登首相（当時）の元秘書...
5,八九年のリクルート事件で、竹下元首相の青木伊平元秘書。


### ELMoで文書ベクトルを算出

In [225]:
register_documents_words_vec = predict_word_vec(elmo_model, register_documents[TEXT_COLUMN].values)
register_documents_vec = calc_doc_vec(register_documents_words_vec)
print("文書ベクトルの次元数確認{shape}".format(shape=register_documents_vec.shape))

2020-03-29 11:36:07,429 INFO: 65 batches, avg len: 70.3
2020-03-29 11:36:10,475 INFO: Finished 1000 sentences.
2020-03-29 11:36:14,301 INFO: Finished 2000 sentences.
2020-03-29 11:36:17,954 INFO: Finished 3000 sentences.
2020-03-29 11:36:21,896 INFO: Finished 4000 sentences.


実行時間:0:00:18.902649
実行時間:0:00:00.166627
各文書のベクトルの次元数確認(4125, 1024)


### Indexを登録する特徴量空間の定義
検索アルゴリズムと必要な初期値はこちら参照<br>
https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

In [0]:
# 登録するデータ1件分の次元数
vector_dim = register_documents_vec.shape[1]
# 分割する領域の数
voronoi_num = 200
# 検索時に含める近傍の領域の数
voronoi_near = 20

#### 高速な近似アルゴリズムIndeIVFFlatを使用
https://github.com/facebookresearch/faiss/wiki/Faster-search

In [0]:
quantizer = faiss.IndexFlatL2(vector_dim)
index = faiss.IndexIVFFlat(quantizer, vector_dim, voronoi_num)

### Indexへの追加（+ボロノイ領域の学習）

In [0]:
assert not index.is_trained
index.train(register_documents_vec)
assert index.is_trained

In [321]:
# idの付与
ids = np.arange(start=1, stop=register_documents_vec.shape[0]+1)
print("idsの長さ{size}".format(size=len(ids)))
ids[:5]

idsの長さ4125


array([1, 2, 3, 4, 5])

In [0]:
index.add_with_ids(register_documents_vec, ids) 

#### 結果の保存

In [0]:
EXPORT_DIR = "./exports"
now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

In [0]:
faiss.write_index(index, EXPORT_DIR + "/features_" + now + ".index")

## 検索エンジンにクエリを投げてみる

#### 質問文の前処理

In [0]:
# 質問文
queries = ["最近傍アルゴリズムの練習にpythonで、類似文章検索エンジンを実装してみました", "野球で強い選手は誰なのか分からない", "コンピュータ", "上野動物園にパンダを見に行きたいと思うが、近くの御徒町が怖い", "野球のルールが分からない"]

In [337]:
# ELMoで単語ベクトルに変換
queries_words_vec = predict_word_vec(elmo_model, queries)
# 文書ベクトルに変換
queries_vec = calc_doc_vec(queries_words_vec)
print("入力の次元{shape}".format(shape=queries_vec.shape))

2020-03-29 12:49:13,061 INFO: 1 batches, avg len: 23.0


実行時間:0:00:00.122881
実行時間:0:00:00.000466
入力の次元(5, 1024)


#### インデックスから類似文を検索する

In [0]:
@time_watch
def search_doc(queries_vec, search_doc_num, index):
    """
    類似文章の検索

    Parameters
    -----
    queries_vec : ndarray
    search_doc_num : int
        何件分まで検索してくるか
    index : index_obj
        faissのindexオブジェクト

    Returns
    -----
    D : D_obj
    I : I_obj

    Notes
    -----
    WARNING : queries_vecは1質問文でないときの戻り値を、ドキュメントで確認中
    """
    d, i = index.search(queries_vec, search_doc_num)
    return d, i

In [339]:
search_doc_num = 10

results_distances, results_ids = search_doc(queries_vec, search_doc_num, index)

実行時間:0:00:00.001804


In [343]:
# 結果の描画
for i, (result_distances, result_ids) in enumerate(zip(results_distances, results_ids)):
    
    print("{i}番目の検索クエリ:{query}".format(i=i+1, query=queries[i]))
    print("-----検索結果-----")

    for j, (result_distance, result_id) in enumerate(zip(result_distances, result_ids)):
        if result_id != -1:
            print("第{j}位\t文書id:{id}\tテキスト:{text}\t距離{distance}"
            .format(j=j+1, id=result_id, text=register_documents[TEXT_COLUMN][result_id], distance=result_distance))
    
    print("-----{i}番目の結果ここまで-----".format(i=i+1))

1番目の検索クエリ:最近傍アルゴリズムの練習にpythonで、類似文章検索エンジンを実装してみました
-----検索結果-----
第1位	文書id:3926	テキスト:アフガニスタン新政権の枠組み協議は、最大民族パシュトゥン人がカギを握っている。	距離16155.263671875
第2位	文書id:181	テキスト:米大リーグ　３４歳・快速左腕が快挙　ヤンキースのデービッド・ウェルズが完全試合	距離20154.412109375
第3位	文書id:839	テキスト:九二年にボーカル・グループ、スーパー・モンキーズの一員として十四歳でデビュー。	距離20227.162109375
第4位	文書id:1638	テキスト:イスラム原理主義運動を扇動するウサマ・ビン・ラディン氏を大使館爆破の黒幕と断定	距離21226.333984375
第5位	文書id:1426	テキスト:ヤンキースタジアムは一九二三年四月にオープンし、大リーグでは四番目に古い球場。	距離21701.05078125
第6位	文書id:2321	テキスト:アフガニスタンのイスラム原理主義勢力タリバンがバーミヤンの仏教遺跡を破壊した。	距離22644.6328125
第7位	文書id:3257	テキスト:野依教授　ノーベル賞までの年譜//イスラエルのウルフ財団からウルフ賞を受賞。	距離22931.46484375
第8位	文書id:975	テキスト:チリはサモラーノ、カメルーンはエムボマ（ガンバ）という決定力のあるエースがいる。	距離25588.87109375
第9位	文書id:1004	テキスト:ワールドカップ（Ｗ杯）フランス大会準優勝国で、２年前のこの大会を制したブラジルや	距離25595.32421875
第10位	文書id:2002	テキスト:新種の猿人化石「アウストラロピテクス・ガルヒ」がエチオピアで発掘されるなど、	距離25642.84375
-----1番目の結果ここまで-----
2番目の検索クエリ:野球で強い選手は誰なのか分からない
-----検索結果-----
第1位	文書id:1039	テキスト:中盤で中田がボールを持ったときは、	距離9064.7802734375
第2位	文書id:985	テキスト:メキシコも個人技のレベルは高いが、	距離9662.720703125
第3位	